In [1]:
%config IPCompleter.use_jedi = False
%pdb off
# %load_ext viztracer
# from viztracer import VizTracer
%load_ext autoreload
%autoreload 3
import sys
from pathlib import Path
import os
from datetime import datetime
from typing import Any, List, Dict, Optional
import numpy as np
import pandas as pd
import globus_sdk
from globus_sdk import AccessTokenAuthorizer, TransferClient, TransferData
from globus_sdk.scopes import TransferScopes

from phoglobushelpers.PhoGlobusHelper import GlobusConnector, KnownEndpoints
from phoglobushelpers.compatibility_objects.Bookmarks import Bookmark, BookmarkList
from phoglobushelpers.compatibility_objects.Files import File, FilesystemDataType, FileList
from phoglobushelpers.compatibility_objects.Tasks import FatalError, Task, TaskList

from phoglobushelpers.path_helpers import convert_filelist_to_new_parent, read_lines_from_file, find_matching_parent_path, known_global_data_root_parent_paths

Automatic pdb calling has been turned OFF


In [2]:
connect_man = GlobusConnector.login_and_get_transfer_client()
transfer_client = connect_man.transfer_client
connect_man.list_endpoints()

Please go to this URL and login: https://auth.globus.org/v2/oauth2/authorize?client_id=769d24e1-d1cc-4198-9ff7-2626485da449&redirect_uri=https%3A%2F%2Fauth.globus.org%2Fv2%2Fweb%2Fauth-code&scope=openid+profile+email+urn%3Aglobus%3Aauth%3Ascope%3Atransfer.api.globus.org%3Aall&state=_default&response_type=code&code_challenge=_MjchyvY0aOFEQwWj9QHduTYmJvlgo_dV2jLhsQbrhU&code_challenge_method=S256&access_type=offline
	 Copied url to clipboard!
My Endpoints:
[c3a90494-2555-11ec-a47d-a50ad076c282] aleinbook-linux
[84991054-07b4-11ed-8d83-a54cf61939f8] Apogee
[6d0251c4-2585-11ec-9e35-3df4ed83d858] Cubix Win10
[c3578f36-42a8-11ec-a50f-b537d6c07c1d] FlatEdge_Server
[af3fcfce-f664-11ed-9a7d-83ef71fbf0ae] LNX00052_Fedora
[20c84240-1eb1-11eb-81b7-0e2f230cc907] Pho Personal Laptop
[560d3a12-be20-11ed-9916-cb2cff506ca5] Pho_Personal_Testix
[debb635c-2556-11ec-a47d-a50ad076c282] rMBP Pink Dot
[e414f584-2556-11ec-a0a7-6b21ca6daf73] rMBP Pink Dot
[41b8fb12-516b-11ee-8a4b-b1038a0991ac] rMBP-PinkDot


In [ ]:
/home/halechr/repos/PhoGlobusHelpers/filelists/GreatGDriveMigration2023/dirsize

# 2023-09-28 - Newest Filelist Transfer

In [4]:
active_root_filelist_parent_path = Path('/nfs/turbo/umms-kdiba/Data/').resolve()

output_filelist_transfer_dict = {
	Path('/nfs/turbo/umms-kdiba/Data/fileList_GreatLakes_pkls_2023-09-27-GL.txt'): Path('/nfs/turbo/umms-kdiba/Data/dest_fileList_LabWorkstation_pkls_2023-09-27-GL.txt'),
	Path('/nfs/turbo/umms-kdiba/Data/fileList_GreatLakes_global_pkls_2023-09-27-GL.txt'): Path('/nfs/turbo/umms-kdiba/Data/dest_fileList_LabWorkstation_global_pkls_2023-09-27-GL.txt'),
	# Path('/nfs/turbo/umms-kdiba/Data/fileList_GreatLakes_HDF5_2023-09-27-GL.txt'): Path('/nfs/turbo/umms-kdiba/Data/dest_fileList_LabWorkstation_HDF5_2023-09-27-GL.txt')
}

output_filelist_transfer_dict

# read the lines from the files
output_file_lines_transfer_list = [(read_lines_from_file(filelist_source_path), read_lines_from_file(filelist_dest_path)) for filelist_source_path, filelist_dest_path in output_filelist_transfer_dict.items()]
output_file_lines_transfer_list

FileNotFoundError: [Errno 2] No such file or directory: '\\nfs\\turbo\\umms-kdiba\\Data\\fileList_GreatLakes_pkls_2023-09-27-GL.txt'

In [ ]:
apogee_data_folder_bookmark = Bookmark(bookmark_id='2a9f2464-01ad-11ee-be0e-195c41bc0be4', name='Apogee Data W', endpoint_id='84991054-07b4-11ed-8d83-a54cf61939f8', path='/~/W/Data/')
lab_Turbo_data_folder_bookmark = Bookmark(bookmark_id='728fb3e8-2597-11ee-80c2-a3018385fcef', name='KDIBA Lab Turbo - Data Global Root Folder', endpoint_id='8c185a84-5c61-4bbc-b12b-11430e20010f', path='/umms-kdiba/Data/')
lab_DataDen_data_folder_bookmark = Bookmark(bookmark_id='431c951e-96c6-11ed-9b93-19370d280681', name='kdiba DataDen Data', endpoint_id='ab65757f-00f5-4e5b-aa21-133187732a01', path='/umms-dibalab/Data/')

In [ ]:

active_transfer_tasks = []
## Perform the transfer:
for filelist_source_path, filelist_dest_path in output_filelist_transfer_dict.items():
	# filelist_source_path = Path('../filelists/fileList_Greatlakes_2023-08-07.txt').resolve()
	src_lines = read_lines_from_file(filelist_source_path)
	# filelist_dest_path = '../filelists/dest_fileList_Apogee_2023-08-07.txt'
	dest_lines = read_lines_from_file(filelist_dest_path)
	
	endpoint_relative_src_lines = [a_line for a_line in src_lines]
	endpoint_relative_dest_lines = [a_line for a_line in dest_lines]
	_transfer_task = connect_man.batch_transfer_files(source_endpoint=lab_Turbo_data_folder_bookmark.endpoint_id, destination_endpoint=lab_Turbo_data_folder_bookmark.endpoint_id, filelist_source=endpoint_relative_src_lines, filelist_dest=endpoint_relative_dest_lines, max_single_file_wait_time_seconds=60.0)
	active_transfer_tasks.append(_transfer_task)
	


In [ ]:
apogee_data_folder_bookmark = Bookmark(bookmark_id='2a9f2464-01ad-11ee-be0e-195c41bc0be4', name='Apogee Data W', endpoint_id='84991054-07b4-11ed-8d83-a54cf61939f8', path='/~/W/Data/')
lab_Turbo_data_folder_bookmark = Bookmark(bookmark_id='728fb3e8-2597-11ee-80c2-a3018385fcef', name='KDIBA Lab Turbo - Data Global Root Folder', endpoint_id='8c185a84-5c61-4bbc-b12b-11430e20010f', path='/umms-kdiba/Data/')

# endpoint_relative_src_lines = [a_line.replace('/umms-kdiba/Data/', '/', 1) for a_line in src_lines]
# endpoint_relative_dest_lines = [a_line.replace('/~/W/Data/', '/', 1) for a_line in dest_lines]

endpoint_relative_src_lines = [a_line for a_line in src_lines]
endpoint_relative_dest_lines = [a_line for a_line in dest_lines]
_transfer_task = connect_man.batch_transfer_files(source_endpoint=lab_Turbo_data_folder_bookmark.endpoint_id, destination_endpoint=apogee_data_folder_bookmark.endpoint_id, filelist_source=endpoint_relative_src_lines, filelist_dest=endpoint_relative_dest_lines, max_single_file_wait_time_seconds=60.0)
_transfer_task


In [ ]:

active_filelist_files = active_root_filelist_parent_path.glob('*.csv')
filelist_dfs_list = []
filelists_list = []
for a_file in active_filelist_files:
    print(f'a_file: {a_file}')
    active_filelist_path = Path(a_file).resolve()
    try:
        user, filename_suffix = active_filelist_path.name.split('_')
        print(f'\t{user}')
        parent_user_folder = Path(f"Data/{user}")
        print(f'\t{parent_user_folder}')
        filelist_df = pd.read_csv(active_filelist_path, header=0, names=["name", "modified_dt", "size_bytes"])
        all_files_list = [parent_user_folder.joinpath(_a_file) for _a_file in filelist_df["name"]]
        filelist_df['name'] = all_files_list
        filelist_dfs_list.append(filelist_df)
        filelists_list.append(all_files_list)
    except pd.errors.ParserError:
        print(f'encountered parser error for {active_filelist_path}')
    except BaseException:
        raise

combined_filelist_df = pd.concat(filelist_dfs_list)
combined_filelist_df


# 2023-09-12 - New Filelist Transfer

In [19]:
from typing import List
from pathlib import Path
import numpy as np
import pandas as pd

def chunk_dataframe(df, size_limit_bytes):
    """
    Splits the dataframe into chunks based on a cumulative size in 'size_bytes' column.

    :param df: DataFrame to split
    :param size_limit_bytes: Max size for each chunk in bytes
    :return: List of dataframes
    """
    chunks = []
    current_chunk = []
    current_chunk_size = 0

    for _, row in df.iterrows():
        if current_chunk_size + row['size_bytes'] > size_limit_bytes:
            chunks.append(pd.DataFrame(current_chunk))
            current_chunk = []
            current_chunk_size = 0

        current_chunk.append(row)
        current_chunk_size += row['size_bytes']

    # Append any remaining data
    if current_chunk:
        chunks.append(pd.DataFrame(current_chunk))

    return chunks

def convert_filelist_to_new_parent(filelist_source: List[Path], original_parent_path: Path = Path(r'/media/MAX/cloud/turbo/Data'), dest_parent_path: Path = Path(r'/media/MAX/Data')):
    """ Converts a list of file paths from their current parent, specified by `original_parent_path`, to their new parent `dest_parent_path` """
    filelist_dest = []
    for path in filelist_source:
        relative_path = str(path.relative_to(original_parent_path))
        new_path = Path(dest_parent_path) / relative_path
        filelist_dest.append(new_path)
    return filelist_dest

# Load filelist from disk
# active_filelist_path = Path('/home/halechr/repo/PhoGlobusHelpers/filelists/session_results_filelist_2023-07-12.csv').resolve()
# active_filelist_path = Path('/home/halechr/repos/PhoGlobusHelpers/filelists/GreatGDriveMigration2023/dirsize/Bapun_dirsize.csv').resolve()
active_root_filelist_parent_path = Path('/home/halechr/Desktop/GreatGDriveMigration2023/dirsize').resolve()
active_filelist_files = active_root_filelist_parent_path.glob('*.csv')
filelist_dfs_list = []
filelists_list = []
for a_file in active_filelist_files:
    print(f'a_file: {a_file}')
    active_filelist_path = Path(a_file).resolve()
    try:
        user, filename_suffix = active_filelist_path.name.split('_')
        print(f'\t{user}')
        parent_user_folder = Path(f"Data/{user}")
        print(f'\t{parent_user_folder}')
        filelist_df = pd.read_csv(active_filelist_path, header=0, names=["name", "modified_dt", "size_bytes"])
        all_files_list = [parent_user_folder.joinpath(_a_file) for _a_file in filelist_df["name"]]
        filelist_df['name'] = all_files_list
        filelist_dfs_list.append(filelist_df)
        filelists_list.append(all_files_list)
    except pd.errors.ParserError:
        print(f'encountered parser error for {active_filelist_path}')
    except BaseException:
        raise

combined_filelist_df = pd.concat(filelist_dfs_list)
combined_filelist_df

a_file: /home/halechr/Desktop/GreatGDriveMigration2023/dirsize/Rachel_dirsize.csv
	Rachel
	Data/Rachel
a_file: /home/halechr/Desktop/GreatGDriveMigration2023/dirsize/Nat_dirsize.csv
	Nat
	Data/Nat
a_file: /home/halechr/Desktop/GreatGDriveMigration2023/dirsize/Utku_dirsize.csv
	Utku
	Data/Utku
a_file: /home/halechr/Desktop/GreatGDriveMigration2023/dirsize/Laurel_dirsize.csv
	Laurel
	Data/Laurel
encountered parser error for /home/halechr/Desktop/GreatGDriveMigration2023/dirsize/Laurel_dirsize.csv
a_file: /home/halechr/Desktop/GreatGDriveMigration2023/dirsize/Kourosh_dirsize.csv
	Kourosh
	Data/Kourosh
a_file: /home/halechr/Desktop/GreatGDriveMigration2023/dirsize/Bapun_dirsize.csv
	Bapun
	Data/Bapun
a_file: /home/halechr/Desktop/GreatGDriveMigration2023/dirsize/Jahngir_dirsize.csv
	Jahngir
	Data/Jahngir
a_file: /home/halechr/Desktop/GreatGDriveMigration2023/dirsize/Hiro_dirsize.csv
	Hiro
	Data/Hiro
a_file: /home/halechr/Desktop/GreatGDriveMigration2023/dirsize/KDIBA_dirsize.csv
	KDIBA
	Da

,name,modified_dt,size_bytes
0,Data/Rachel/Take 2021-11-24 11.23.05 AM.csv,2022-08-11 12:41:13,1888355
1,Data/Rachel/Take 2021-11-24 11.13.41 AM.csv,2022-08-11 12:40:48,731099
2,Data/Rachel/Take 2021-11-24 11.12.49 AM.csv,2022-08-11 12:39:44,13590
3,Data/Rachel/Take 2021-08-27 12.59.15 PM.tak,2022-02-25 13:09:10,10307644
4,Data/Rachel/Take 2021-11-24 11.34.03 AM.tak,2022-02-25 13:00:58,262879235
...,...,...,...
690,Data/Output/2023-06-01/kdiba/vvp01/two/2006-4-...,2023-06-01 02:09:10,537957
691,Data/Output/2023-06-01/kdiba/vvp01/two/2006-4-...,2023-06-01 02:09:08,748944
692,Data/Output/2023-06-01/kdiba/vvp01/two/2006-4-...,2023-06-01 02:09:03,735368
693,Data/Output/2023-06-01/kdiba/vvp01/two/2006-4-...,2023-06-01 02:08:58,500886


In [26]:
total_all_files_num_GB = combined_filelist_df.size_bytes.sum()/1e9
total_all_files_num_GB # 65467.555576481

65467.555576481

In [ ]:
size_limit = 750e9  # 750GB in bytes
size_limit = 3999e9  # 4TB in bytes
chunks = chunk_dataframe(combined_filelist_df, size_limit)
num_chunks = len(chunks)
print(f'num_chunks: {num_chunks}')
# for chunk in chunks:
#     print(chunk)

,name,modified_dt,size_bytes
0,Data/Rachel/Take 2021-11-24 11.23.05 AM.csv,2022-08-11 12:41:13,1888355
1,Data/Rachel/Take 2021-11-24 11.13.41 AM.csv,2022-08-11 12:40:48,731099
2,Data/Rachel/Take 2021-11-24 11.12.49 AM.csv,2022-08-11 12:39:44,13590
3,Data/Rachel/Take 2021-08-27 12.59.15 PM.tak,2022-02-25 13:09:10,10307644
4,Data/Rachel/Take 2021-11-24 11.34.03 AM.tak,2022-02-25 13:00:58,262879235
...,...,...,...
690,Data/Output/2023-06-01/kdiba/vvp01/two/2006-4-...,2023-06-01 02:09:10,537957
691,Data/Output/2023-06-01/kdiba/vvp01/two/2006-4-...,2023-06-01 02:09:08,748944
692,Data/Output/2023-06-01/kdiba/vvp01/two/2006-4-...,2023-06-01 02:09:03,735368
693,Data/Output/2023-06-01/kdiba/vvp01/two/2006-4-...,2023-06-01 02:08:58,500886


In [ ]:

filelist_df = pd.read_csv(active_filelist_path, header=0, names=["name", "modified_dt", "size_bytes"])
filelist_df


In [9]:
np.sum(filelist_df['size_bytes']/(1024.0 * 1024.0)) # Convert to GigaBytes (GB)



21955757.33845806

In [6]:
filelist_df.dtypes

name           object
modified_dt    object
size_bytes      int64
dtype: object

In [ ]:
filelist_source = [Path(a_path_str).resolve() for a_path_str in filelist_df.Path]



source_parent_path = Path(r'/media/MAX/cloud/turbo/Data')
dest_parent_path = Path(r'/media/MAX/Data')
# # Build the destination filelist from the source_filelist and the two paths:
filelist_dest = convert_filelist_to_new_parent(filelist_source, original_parent_path=source_parent_path, dest_parent_path=dest_parent_path)
filelist_dest

# filelist_source
# filelist_dest